In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install numpy scipy
!pip install matplotlib
!pip install tensorflow
!pip install -U scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 41.5 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


In [49]:
import matplotlib
import sys
sys.path.append('/content/drive/MyDrive/Age_Detection_Project/')

from model.ResNet50 import AgeEstimatorModel as ResNetModel
from model.EfficientNetB0 import AgeEstimatorModel1 as EfficientNetModel
from model.VGGNet import AgeEstimatorModel3 as VGGNetModel
from model.AlexNet import AlexNetAgeDetection as AlexNetModel
from model.CNNmodel import AgeEstimatorModel5 as CNNModel
from model.GoogleNetV2 import inceptionv4
import tensorflow as tf
matplotlib.use("Agg")
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import numpy as np
import random
import cv2
import glob

In [6]:
def train_model(model, model_name):
  # compile the model
  opt = tf.keras.optimizers.legacy.Adam(learning_rate=lr)
  model.compile(loss="mean_squared_error", optimizer=opt, metrics=["mae"])  # using mean squared error loss

  # train the model
  H = model.fit(aug.flow(trainX, trainY, batch_size=batch_size),
                validation_data=(testX, testY),
                steps_per_epoch=len(trainX) // batch_size,
                epochs=epochs, verbose=1)

  # save the model to disk
  model.save(f"{model_name}_age_detection.model")

  return H

In [53]:
def plot_model_performance(H, model_name):
    # plot training/validation loss/mae
    plt.style.use("ggplot")
    plt.figure()
    N = epochs
    plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
    plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
    plt.title(f"{model_name} Loss")
    plt.xlabel("Epoch #")
    plt.ylim([0,2000])
    plt.ylabel("Loss")
    plt.legend(loc="upper right")
    plt.savefig(f"{model_name}_loss_plot.png")

    #plt.style.use("ggplot")
    plt.figure()
    N = epochs
    plt.plot(np.arange(0, N), H.history["mae"], label="train_mae")
    plt.plot(np.arange(0, N), H.history["val_mae"], label="val_mae")
    plt.title(f"{model_name} MAE")
    plt.xlabel("Epoch #")
    plt.ylim([0,100])
    plt.ylabel("MAE")
    plt.legend(loc="upper right")
    plt.savefig(f"{model_name}_mae_plot.png")

    plt.figure()
    N = epochs
    plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
    plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
    plt.plot(np.arange(0, N), H.history["mae"], label="train_mae")
    plt.plot(np.arange(0, N), H.history["val_mae"], label="val_mae")
    plt.title(f"{model_name} Loss & MAE")
    plt.xlabel("Epoch #")
    plt.ylabel("MAE")
    plt.ylabel("Loss")
    plt.ylim([0,2000])
    plt.legend(loc="upper right")
    plt.savefig(f"{model_name}_plot.png")


In [8]:
# Paths and parameters initialization
dataset_path = "/content/drive/MyDrive/Age_Detection_Project/Faces"
model_path_resnet = "/content/drive/MyDrive/Age_Detection_Project/resnet_age_detection.model"
model_path_effnet = "/content/drive/MyDrive/Age_Detection_Project/effnet_age_detection.model"
model_path_alexnet = "/content/drive/MyDrive/Age_Detection_Project/AlexNet2_age_detection.model"
model_path_vggnet = "/content/drive/MyDrive/Age_Detection_Project/VGGNet_0.001_age_detection.model"
model_path_googlenet = "/content/drive/MyDrive/Age_Detection_Project/GoogleNet_age_detection.model"
model_path_cnn = "/content/drive/MyDrive/Age_Detection_Project/CNNModel_age_detection.model"
plot_path_resnet = "/content/drive/MyDrive/Age_Detection_Project/resnet_plot.png"
plot_path_effnet = "/content/drive/MyDrive/Age_Detection_Project/effnet_plot.png"
plot_path_googlenet = "/content/drive/MyDrive/Age_Detection_Project/googlenet_plot.png"
plot_path_cnn = "/content/drive/MyDrive/Age_Detection_Project/cnn_plot.png"
plot_path_alexnet = "/content/drive/MyDrive/Age_Detection_Project/alexnet_plot.png"
plot_path_vggnet = "/content/drive/MyDrive/Age_Detection_Project/vggnet_plot.png"

In [ ]:
# Paths and parameters initialization for local
dataset_path = "Faces"
model_path_resnet = "resnet_age_detection.model"
model_path_effnet = "effnet_age_detection.model"
model_path_googlenet = "googlenet_age_detection.model"
model_path_vggnet = "vggnet_age_detection.model"
model_path_cnn = "cnn_age_detection.model"
model_path_alexnet = "alexnet_age_detection.model"
plot_path_resnet = "resnet_plot.png"
plot_path_effnet = "effnet_plot.png"
plot_path_googlenet = "googlenet_plot.png"
plot_path_alexnet = "alexnet_plot.png"
plot_path_vggnet = "vggnet_plot.png"
plot_path_cnn = "cnn_plot.png"
plot_path_alexnet2 = "alexnet2_plot.png"

In [9]:
# Initial parameters
epochs = 50
lr = 0.001
batch_size = 64
img_dims = (96, 96, 3)

# Load and preprocess the data
data = []
age_labels = []

# Load image files from the dataset
image_files = [f for f in glob.glob(dataset_path + "/*.jpg", recursive=True)]

print(len(image_files))

random.seed(42)
random.shuffle(image_files)

# Create ground-truth labels from the image paths

for i in range(0,2000):
    image = cv2.imread(image_files[i])
    image = cv2.resize(image, (img_dims[0], img_dims[1]))
    image = img_to_array(image)
    data.append(image)

    # Extract age labels from the image paths
    label = int(image_files[i].split("/")[-1].split("_")[0])
    age_labels.append(label)

print(len(data))
print(age_labels)


4521
2000
[27, 65, 21, 30, 22, 8, 26, 16, 32, 81, 34, 32, 49, 65, 42, 42, 34, 37, 41, 32, 5, 56, 32, 45, 34, 24, 11, 2, 1, 25, 28, 1, 52, 38, 24, 56, 54, 36, 26, 45, 15, 80, 72, 7, 36, 26, 35, 6, 3, 33, 40, 2, 14, 29, 24, 90, 67, 4, 46, 26, 29, 23, 6, 38, 55, 55, 20, 41, 21, 10, 20, 31, 42, 34, 7, 5, 28, 25, 28, 53, 5, 54, 49, 43, 72, 30, 26, 37, 36, 76, 31, 66, 17, 49, 27, 60, 26, 5, 1, 1, 50, 41, 20, 26, 32, 14, 80, 54, 8, 16, 2, 38, 26, 37, 55, 1, 4, 27, 30, 73, 8, 35, 32, 24, 31, 27, 36, 15, 85, 40, 56, 32, 13, 54, 28, 28, 33, 38, 89, 80, 26, 1, 25, 25, 26, 65, 61, 63, 22, 9, 30, 28, 68, 39, 28, 13, 26, 43, 21, 35, 21, 52, 1, 29, 33, 24, 35, 54, 29, 73, 26, 3, 55, 4, 27, 53, 4, 28, 27, 26, 27, 50, 39, 75, 32, 74, 29, 15, 72, 4, 81, 1, 67, 35, 85, 36, 80, 70, 50, 5, 30, 4, 9, 76, 2, 37, 24, 30, 30, 1, 3, 1, 45, 30, 35, 72, 32, 34, 1, 33, 26, 10, 78, 25, 35, 28, 27, 45, 26, 78, 49, 27, 90, 2, 56, 52, 45, 26, 24, 26, 36, 28, 60, 62, 21, 1, 39, 63, 26, 15, 26, 11, 28, 80, 35, 27, 26, 4

In [10]:
# Pre-process the data
data = np.array(data, dtype="float") / 255.0
labels = np.array(age_labels)


print("Data shape:", data.shape)
print("Labels shape:", labels.shape)

# Split dataset for training and validation
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.2, random_state=42)

# Augmenting dataset
aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.1,
                         height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
                         horizontal_flip=True, fill_mode="nearest")


Data shape: (2000, 96, 96, 3)
Labels shape: (2000,)


In [11]:
# build and train ResNet50 model
model_resnet = ResNetModel.build(width=img_dims[0], height=img_dims[1], depth=img_dims[2], classes=1)
H_resnet = train_model(model_resnet, "ResNet50")


94765736/94765736 [==============================] - 3s 0us/step
Epoch 1/50
25/25 [==============================] - 16s 225ms/step - loss: 504.5963 - mae: 17.0787 - val_loss: 1080.9764 - val_mae: 26.0793
Epoch 2/50
25/25 [==============================] - 4s 150ms/step - loss: 309.3094 - mae: 13.5553 - val_loss: 1117.1705 - val_mae: 26.5949
Epoch 3/50
25/25 [==============================] - 4s 152ms/step - loss: 278.9889 - mae: 12.7714 - val_loss: 537.5106 - val_mae: 19.6135
Epoch 4/50
25/25 [==============================] - 4s 151ms/step - loss: 237.3659 - mae: 11.5680 - val_loss: 1168.0945 - val_mae: 27.3198
Epoch 5/50
25/25 [==============================] - 4s 156ms/step - loss: 213.7020 - mae: 11.0801 - val_loss: 812.9966 - val_mae: 22.0001
Epoch 6/50
25/25 [==============================] - 4s 153ms/step - loss: 191.3987 - mae: 10.2504 - val_loss: 754.6967 - val_mae: 21.0553
Epoch 7/50
25/25 [==============================] - 4s 150ms/step - loss: 186.5336 - mae: 10.2736 - val

In [13]:
plot_model_performance(H_resnet, "ResNet50")

In [16]:
# build and train EfficientNetB0 model
model_effnet = EfficientNetModel.build(width=img_dims[0], height=img_dims[1], depth=img_dims[2], classes=1)
H_effnet = train_model(model_effnet, "EfficientNetB0")


16705208/16705208 [==============================] - 1s 0us/step
Epoch 1/50
25/25 [==============================] - 15s 243ms/step - loss: 663.6571 - mae: 19.9847 - val_loss: 2232.9980 - val_mae: 41.6549
Epoch 2/50
25/25 [==============================] - 4s 149ms/step - loss: 309.8270 - mae: 13.6314 - val_loss: 1793.6447 - val_mae: 35.9915
Epoch 3/50
25/25 [==============================] - 4s 146ms/step - loss: 251.6504 - mae: 12.2437 - val_loss: 1118.2538 - val_mae: 26.6125
Epoch 4/50
25/25 [==============================] - 4s 152ms/step - loss: 208.3811 - mae: 11.1030 - val_loss: 1240.9766 - val_mae: 28.3312
Epoch 5/50
25/25 [==============================] - 4s 148ms/step - loss: 176.5812 - mae: 10.1601 - val_loss: 861.3080 - val_mae: 22.7824
Epoch 6/50
25/25 [==============================] - 4s 147ms/step - loss: 162.4234 - mae: 9.6500 - val_loss: 631.9753 - val_mae: 18.7770
Epoch 7/50
25/25 [==============================] - 4s 147ms/step - loss: 138.9562 - mae: 8.8850 - val_

In [21]:
plot_model_performance(H_effnet, "EfficientNetB0")

In [50]:
# Initial parameters
epochs = 50
lr = 0.001
batch_size = 64
img_dims = (160, 160, 3)

# Load and preprocess the data
data = []
age_labels = []

# Load image files from the dataset
image_files = [f for f in glob.glob(dataset_path + "/*.jpg", recursive=True)]

print(len(image_files))

random.seed(42)
random.shuffle(image_files)

# Create ground-truth labels from the image paths

for i in range(0,2000):
    image = cv2.imread(image_files[i])
    image = cv2.resize(image, (img_dims[0], img_dims[1]))
    image = img_to_array(image)
    data.append(image)

    # Extract age labels from the image paths
    label = int(image_files[i].split("/")[-1].split("_")[0])
    age_labels.append(label)

print(len(data))
print(age_labels)


4521
2000
[27, 65, 21, 30, 22, 8, 26, 16, 32, 81, 34, 32, 49, 65, 42, 42, 34, 37, 41, 32, 5, 56, 32, 45, 34, 24, 11, 2, 1, 25, 28, 1, 52, 38, 24, 56, 54, 36, 26, 45, 15, 80, 72, 7, 36, 26, 35, 6, 3, 33, 40, 2, 14, 29, 24, 90, 67, 4, 46, 26, 29, 23, 6, 38, 55, 55, 20, 41, 21, 10, 20, 31, 42, 34, 7, 5, 28, 25, 28, 53, 5, 54, 49, 43, 72, 30, 26, 37, 36, 76, 31, 66, 17, 49, 27, 60, 26, 5, 1, 1, 50, 41, 20, 26, 32, 14, 80, 54, 8, 16, 2, 38, 26, 37, 55, 1, 4, 27, 30, 73, 8, 35, 32, 24, 31, 27, 36, 15, 85, 40, 56, 32, 13, 54, 28, 28, 33, 38, 89, 80, 26, 1, 25, 25, 26, 65, 61, 63, 22, 9, 30, 28, 68, 39, 28, 13, 26, 43, 21, 35, 21, 52, 1, 29, 33, 24, 35, 54, 29, 73, 26, 3, 55, 4, 27, 53, 4, 28, 27, 26, 27, 50, 39, 75, 32, 74, 29, 15, 72, 4, 81, 1, 67, 35, 85, 36, 80, 70, 50, 5, 30, 4, 9, 76, 2, 37, 24, 30, 30, 1, 3, 1, 45, 30, 35, 72, 32, 34, 1, 33, 26, 10, 78, 25, 35, 28, 27, 45, 26, 78, 49, 27, 90, 2, 56, 52, 45, 26, 24, 26, 36, 28, 60, 62, 21, 1, 39, 63, 26, 15, 26, 11, 28, 80, 35, 27, 26, 4

In [51]:
# Pre-process the data
data = np.array(data, dtype="float") / 255.0
labels = np.array(age_labels)


print("Data shape:", data.shape)
print("Labels shape:", labels.shape)

# Split dataset for training and validation
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.2, random_state=42)

# Augmenting dataset
aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.1,
                         height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
                         horizontal_flip=True, fill_mode="nearest")


Data shape: (2000, 160, 160, 3)
Labels shape: (2000,)


In [52]:
# build and train GoogleNet model
model_googlenet = inceptionv4((img_dims[0], img_dims[1], img_dims[2]), dropout_keep=0.8)
H_googlenet = train_model(model_googlenet, "GoogleNet")


Epoch 1/50
25/25 [==============================] - 14s 378ms/step - loss: 621.9210 - mae: 19.3429 - val_loss: 1584.3164 - val_mae: 32.9525
Epoch 2/50
25/25 [==============================] - 8s 330ms/step - loss: 461.4732 - mae: 16.5425 - val_loss: 1511.3788 - val_mae: 31.9628
Epoch 3/50
25/25 [==============================] - 9s 345ms/step - loss: 448.0006 - mae: 16.3592 - val_loss: 1503.0916 - val_mae: 31.8501
Epoch 4/50
25/25 [==============================] - 8s 336ms/step - loss: 445.9298 - mae: 16.2099 - val_loss: 1121.2727 - val_mae: 26.9378
Epoch 5/50
25/25 [==============================] - 8s 334ms/step - loss: 416.1859 - mae: 15.6063 - val_loss: 1206.6901 - val_mae: 28.1006
Epoch 6/50
25/25 [==============================] - 9s 337ms/step - loss: 384.4840 - mae: 15.2172 - val_loss: 1142.8491 - val_mae: 27.2938
Epoch 7/50
25/25 [==============================] - 9s 339ms/step - loss: 382.8388 - mae: 15.0424 - val_loss: 1153.3472 - val_mae: 27.4671
Epoch 8/50
25/25 [========

In [54]:
plot_model_performance(H_googlenet, "GoogleNet")

In [23]:
# build and train VGGNet model
model_vggnet = VGGNetModel.build(width=img_dims[0], height=img_dims[1], depth=img_dims[2], classes=1)
H_vggnet = train_model(model_vggnet, "VGGNet")


58889256/58889256 [==============================] - 2s 0us/step
Epoch 1/50
25/25 [==============================] - 8s 186ms/step - loss: 1486.0374 - mae: 31.7234 - val_loss: 1158.6910 - val_mae: 27.1863
Epoch 2/50
25/25 [==============================] - 4s 143ms/step - loss: 535.8777 - mae: 17.9759 - val_loss: 495.9633 - val_mae: 17.9345
Epoch 3/50
25/25 [==============================] - 4s 143ms/step - loss: 429.7108 - mae: 16.1038 - val_loss: 493.4709 - val_mae: 17.1152
Epoch 4/50
25/25 [==============================] - 4s 142ms/step - loss: 424.0559 - mae: 15.9662 - val_loss: 478.3147 - val_mae: 16.7163
Epoch 5/50
25/25 [==============================] - 4s 142ms/step - loss: 421.7945 - mae: 15.8726 - val_loss: 456.1532 - val_mae: 16.7855
Epoch 6/50
25/25 [==============================] - 4s 148ms/step - loss: 411.9605 - mae: 15.6987 - val_loss: 459.7102 - val_mae: 16.5421
Epoch 7/50
25/25 [==============================] - 4s 142ms/step - loss: 408.1403 - mae: 15.6915 - val_l

In [25]:
plot_model_performance(H_vggnet, "VGGNet")

In [27]:
# Initial parameters
epochs = 50
lr = 0.1
batch_size = 64
img_dims_AlexNet2 = (227, 227, 3)

# Load and preprocess the data
data_AlexNet2 = []
age_labels = []

# Load image files from the dataset
image_files = [f for f in glob.glob(dataset_path + "/*.jpg", recursive=True)]

print(len(image_files))

random.seed(42)
random.shuffle(image_files)

# Create ground-truth labels from the image paths
for i in range(0,2000):
    image = cv2.imread(image_files[i])
    image = cv2.resize(image, (img_dims_AlexNet2[0], img_dims_AlexNet2[1]))
    image = img_to_array(image)
    data_AlexNet2.append(image)

    # Extract age labels from the image paths
    label = int(image_files[i].split("/")[-1].split("_")[0])
    age_labels.append(label)

print(len(data_AlexNet2))
print(age_labels)



4521
2000
[27, 65, 21, 30, 22, 8, 26, 16, 32, 81, 34, 32, 49, 65, 42, 42, 34, 37, 41, 32, 5, 56, 32, 45, 34, 24, 11, 2, 1, 25, 28, 1, 52, 38, 24, 56, 54, 36, 26, 45, 15, 80, 72, 7, 36, 26, 35, 6, 3, 33, 40, 2, 14, 29, 24, 90, 67, 4, 46, 26, 29, 23, 6, 38, 55, 55, 20, 41, 21, 10, 20, 31, 42, 34, 7, 5, 28, 25, 28, 53, 5, 54, 49, 43, 72, 30, 26, 37, 36, 76, 31, 66, 17, 49, 27, 60, 26, 5, 1, 1, 50, 41, 20, 26, 32, 14, 80, 54, 8, 16, 2, 38, 26, 37, 55, 1, 4, 27, 30, 73, 8, 35, 32, 24, 31, 27, 36, 15, 85, 40, 56, 32, 13, 54, 28, 28, 33, 38, 89, 80, 26, 1, 25, 25, 26, 65, 61, 63, 22, 9, 30, 28, 68, 39, 28, 13, 26, 43, 21, 35, 21, 52, 1, 29, 33, 24, 35, 54, 29, 73, 26, 3, 55, 4, 27, 53, 4, 28, 27, 26, 27, 50, 39, 75, 32, 74, 29, 15, 72, 4, 81, 1, 67, 35, 85, 36, 80, 70, 50, 5, 30, 4, 9, 76, 2, 37, 24, 30, 30, 1, 3, 1, 45, 30, 35, 72, 32, 34, 1, 33, 26, 10, 78, 25, 35, 28, 27, 45, 26, 78, 49, 27, 90, 2, 56, 52, 45, 26, 24, 26, 36, 28, 60, 62, 21, 1, 39, 63, 26, 15, 26, 11, 28, 80, 35, 27, 26, 4

In [28]:
# Pre-process the data
data_AlexNet2 = np.array(data_AlexNet2, dtype="float") / 255.0
labels = np.array(age_labels)

print("Data shape:", data_AlexNet2.shape)
print("Labels shape:", labels.shape)

# Split dataset for training and validation
(trainX, testX, trainY, testY) = train_test_split(data_AlexNet2, labels, test_size=0.2, random_state=42)

# Augmenting dataset
aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.1,
                         height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
                         horizontal_flip=True, fill_mode="nearest")


Data shape: (2000, 227, 227, 3)
Labels shape: (2000,)


In [29]:
# build and train AlexNet model
model_alexnet2 = AlexNetModel.build(width=img_dims_AlexNet2[0], height=img_dims_AlexNet2[1], depth=img_dims_AlexNet2[2], classes=1)
H_alexnet2 = train_model(model_alexnet2, "AlexNet2")


Epoch 1/50
25/25 [==============================] - 19s 662ms/step - loss: 732.9800 - mae: 20.8247 - val_loss: 501242265600.0000 - val_mae: 685370.0625
Epoch 2/50
25/25 [==============================] - 16s 658ms/step - loss: 461.8248 - mae: 16.8238 - val_loss: 663392.3125 - val_mae: 649.2473
Epoch 3/50
25/25 [==============================] - 16s 658ms/step - loss: 449.0999 - mae: 16.5166 - val_loss: 32181.0020 - val_mae: 155.4067
Epoch 4/50
25/25 [==============================] - 16s 646ms/step - loss: 420.9516 - mae: 16.0418 - val_loss: 7570.7739 - val_mae: 79.7171
Epoch 5/50
25/25 [==============================] - 16s 653ms/step - loss: 439.4980 - mae: 16.3253 - val_loss: 5744.5464 - val_mae: 66.7153
Epoch 6/50
25/25 [==============================] - 17s 669ms/step - loss: 432.5973 - mae: 16.1196 - val_loss: 341466.6875 - val_mae: 551.1375
Epoch 7/50
25/25 [==============================] - 17s 662ms/step - loss: 453.6521 - mae: 16.6339 - val_loss: 1345950.5000 - val_mae: 1069.

In [34]:
matplotlib.pyplot.close()

In [35]:
plot_model_performance(H_alexnet2, "AlexNet2")

<ipython-input-32-4dd96dd8f170>:4: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure()


In [42]:
# Initial parameters
epochs = 50
lr = 0.1
batch_size = 64
img_dims_CNN = (200, 200, 3)

# Load and preprocess the data
data_CNN = []
age_labels = []

# Load image files from the dataset
image_files = [f for f in glob.glob(dataset_path + "/*.jpg", recursive=True)]

print(len(image_files))

random.seed(42)
random.shuffle(image_files)

# Create ground-truth labels from the image paths
for i in range(0,2000):
    image = cv2.imread(image_files[i])
    image = cv2.resize(image, (img_dims_CNN[0], img_dims_CNN[1]))
    image = img_to_array(image)
    data_CNN.append(image)

    # Extract age labels from the image paths
    label = int(image_files[i].split("/")[-1].split("_")[0])
    age_labels.append(label)

print(len(data_CNN))
print(age_labels)


# Pre-process the data
data_CNN = np.array(data_CNN, dtype="float") / 255.0
labels = np.array(age_labels)

print("Data shape:", data_CNN.shape)
print("Labels shape:", labels.shape)

# Split dataset for training and validation
(trainX, testX, trainY, testY) = train_test_split(data_CNN, labels, test_size=0.2, random_state=42)

# Augmenting dataset
aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.1,
                         height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
                         horizontal_flip=True, fill_mode="nearest")


4521
2000
[27, 65, 21, 30, 22, 8, 26, 16, 32, 81, 34, 32, 49, 65, 42, 42, 34, 37, 41, 32, 5, 56, 32, 45, 34, 24, 11, 2, 1, 25, 28, 1, 52, 38, 24, 56, 54, 36, 26, 45, 15, 80, 72, 7, 36, 26, 35, 6, 3, 33, 40, 2, 14, 29, 24, 90, 67, 4, 46, 26, 29, 23, 6, 38, 55, 55, 20, 41, 21, 10, 20, 31, 42, 34, 7, 5, 28, 25, 28, 53, 5, 54, 49, 43, 72, 30, 26, 37, 36, 76, 31, 66, 17, 49, 27, 60, 26, 5, 1, 1, 50, 41, 20, 26, 32, 14, 80, 54, 8, 16, 2, 38, 26, 37, 55, 1, 4, 27, 30, 73, 8, 35, 32, 24, 31, 27, 36, 15, 85, 40, 56, 32, 13, 54, 28, 28, 33, 38, 89, 80, 26, 1, 25, 25, 26, 65, 61, 63, 22, 9, 30, 28, 68, 39, 28, 13, 26, 43, 21, 35, 21, 52, 1, 29, 33, 24, 35, 54, 29, 73, 26, 3, 55, 4, 27, 53, 4, 28, 27, 26, 27, 50, 39, 75, 32, 74, 29, 15, 72, 4, 81, 1, 67, 35, 85, 36, 80, 70, 50, 5, 30, 4, 9, 76, 2, 37, 24, 30, 30, 1, 3, 1, 45, 30, 35, 72, 32, 34, 1, 33, 26, 10, 78, 25, 35, 28, 27, 45, 26, 78, 49, 27, 90, 2, 56, 52, 45, 26, 24, 26, 36, 28, 60, 62, 21, 1, 39, 63, 26, 15, 26, 11, 28, 80, 35, 27, 26, 4

In [43]:
# build and train CNN model
model_cnn = CNNModel.build(width=img_dims_CNN[0], height=img_dims_CNN[1], depth=img_dims_CNN[2], classes=1)
H_cnn = train_model(model_cnn, "CNNModel")


Epoch 1/50
25/25 [==============================] - 14s 493ms/step - loss: 50323050496.0000 - mae: 42930.7383 - val_loss: 525.8964 - val_mae: 17.0470
Epoch 2/50
25/25 [==============================] - 12s 493ms/step - loss: 438.7573 - mae: 16.1002 - val_loss: 497.9506 - val_mae: 17.5820
Epoch 3/50
25/25 [==============================] - 12s 495ms/step - loss: 426.9929 - mae: 16.1210 - val_loss: 513.6060 - val_mae: 17.0159
Epoch 4/50
25/25 [==============================] - 12s 492ms/step - loss: 430.8368 - mae: 16.0337 - val_loss: 497.8303 - val_mae: 17.5572
Epoch 5/50
25/25 [==============================] - 12s 489ms/step - loss: 426.1012 - mae: 16.0465 - val_loss: 498.5014 - val_mae: 17.6588
Epoch 6/50
25/25 [==============================] - 12s 495ms/step - loss: 426.1451 - mae: 16.0264 - val_loss: 499.3453 - val_mae: 17.2393
Epoch 7/50
25/25 [==============================] - 12s 490ms/step - loss: 427.3448 - mae: 16.0584 - val_loss: 498.1459 - val_mae: 17.3345
Epoch 8/50
25/25

In [45]:
plot_model_performance(H_cnn, "CNNModel")

In [56]:
def test_model(model, model_name):
  total_absolute_error = 0

  for i in range(0,100):
    random_image_path = random.choice(image_files)
    random_image = cv2.imread(random_image_path)

    #Adapt for each models (96,96) for VGGNet, EfficientNet, ResNet / (227,227) for AlexNet / (200,200) for CNN / (160,160) for GoogleNet
    random_image = cv2.resize(random_image, (160, 160))

    # Extract the actual age from the image path
    actual_age = int(random_image_path.split("/")[-1].split("_")[0])

    # Preprocess the image for prediction
    random_image = random_image.astype("float") / 255.0
    random_image = img_to_array(random_image)
    random_image = np.expand_dims(random_image, axis=0)

    predicted_age = model.predict(random_image)

    # Calculate absolute error
    absolute_error = np.abs(predicted_age[0][0] - actual_age)
    total_absolute_error += absolute_error
    # Display the actual age and model's prediction
    #print(f"Model: {model_name}")
    print(f"Actual age: {actual_age}")
    print(f"Predicted age: {int(predicted_age[0][0])}\n")

  total_absolute_error = total_absolute_error / 100
  print(total_absolute_error)


In [15]:
# Testing ResNet50 model
test_model(model_resnet, "ResNet50")

1/1 [==============================] - 1s 1s/step
Actual age: 52
Predicted age: 27

1/1 [==============================] - 0s 22ms/step
Actual age: 17
Predicted age: 14

1/1 [==============================] - 0s 21ms/step
Actual age: 67
Predicted age: 24

1/1 [==============================] - 0s 21ms/step
Actual age: 40
Predicted age: 29

1/1 [==============================] - 0s 22ms/step
Actual age: 60
Predicted age: 21

1/1 [==============================] - 0s 21ms/step
Actual age: 21
Predicted age: 22

1/1 [==============================] - 0s 21ms/step
Actual age: 26
Predicted age: 17

1/1 [==============================] - 0s 21ms/step
Actual age: 61
Predicted age: 37

1/1 [==============================] - 0s 21ms/step
Actual age: 31
Predicted age: 15

1/1 [==============================] - 0s 21ms/step
Actual age: 36
Predicted age: 41

1/1 [==============================] - 0s 22ms/step
Actual age: 23
Predicted age: 21

1/1 [==============================] - 0s 22ms/step
Actu

In [22]:
# Testing EfficientNetB0 model
test_model(model_effnet, "EfficientNetB0")

1/1 [==============================] - 2s 2s/step
Actual age: 28
Predicted age: 27

1/1 [==============================] - 0s 28ms/step
Actual age: 24
Predicted age: 28

1/1 [==============================] - 0s 25ms/step
Actual age: 27
Predicted age: 21

1/1 [==============================] - 0s 25ms/step
Actual age: 8
Predicted age: 27

1/1 [==============================] - 0s 25ms/step
Actual age: 28
Predicted age: 26

1/1 [==============================] - 0s 24ms/step
Actual age: 1
Predicted age: 27

1/1 [==============================] - 0s 24ms/step
Actual age: 27
Predicted age: 24

1/1 [==============================] - 0s 24ms/step
Actual age: 26
Predicted age: 24

1/1 [==============================] - 0s 25ms/step
Actual age: 25
Predicted age: 20

1/1 [==============================] - 0s 24ms/step
Actual age: 64
Predicted age: 28

1/1 [==============================] - 0s 25ms/step
Actual age: 45
Predicted age: 19

1/1 [==============================] - 0s 25ms/step
Actual

In [59]:
# Testing GoogleNet model
test_model(model_googlenet, "GoogleNet")

1/1 [==============================] - 0s 20ms/step
Actual age: 7
Predicted age: 1

1/1 [==============================] - 0s 19ms/step
Actual age: 8
Predicted age: 1

1/1 [==============================] - 0s 19ms/step
Actual age: 6
Predicted age: 1

1/1 [==============================] - 0s 21ms/step
Actual age: 36
Predicted age: 1

1/1 [==============================] - 0s 21ms/step
Actual age: 26
Predicted age: 1

1/1 [==============================] - 0s 20ms/step
Actual age: 63
Predicted age: 1

1/1 [==============================] - 0s 20ms/step
Actual age: 35
Predicted age: 1

1/1 [==============================] - 0s 20ms/step
Actual age: 23
Predicted age: 1

1/1 [==============================] - 0s 20ms/step
Actual age: 22
Predicted age: 1

1/1 [==============================] - 0s 19ms/step
Actual age: 14
Predicted age: 1

1/1 [==============================] - 0s 19ms/step
Actual age: 26
Predicted age: 1

1/1 [==============================] - 0s 22ms/step
Actual age: 1
Pr

In [40]:
# Testing AlexNet model
test_model(model_alexnet2, "AlexNet")

1/1 [==============================] - 0s 322ms/step
Actual age: 22
Predicted age: 31

1/1 [==============================] - 0s 19ms/step
Actual age: 11
Predicted age: 34

1/1 [==============================] - 0s 20ms/step
Actual age: 55
Predicted age: 27

1/1 [==============================] - 0s 20ms/step
Actual age: 54
Predicted age: 34

1/1 [==============================] - 0s 20ms/step
Actual age: 26
Predicted age: 30

1/1 [==============================] - 0s 20ms/step
Actual age: 21
Predicted age: 31

1/1 [==============================] - 0s 20ms/step
Actual age: 75
Predicted age: 36

1/1 [==============================] - 0s 20ms/step
Actual age: 38
Predicted age: 28

1/1 [==============================] - 0s 19ms/step
Actual age: 26
Predicted age: 39

1/1 [==============================] - 0s 20ms/step
Actual age: 26
Predicted age: 34

1/1 [==============================] - 0s 19ms/step
Actual age: 62
Predicted age: 25

1/1 [==============================] - 0s 19ms/step
A

In [26]:
# Testing VGGNet model
test_model(model_vggnet, "VGGNet")

1/1 [==============================] - 0s 407ms/step
Actual age: 1
Predicted age: 1

1/1 [==============================] - 0s 20ms/step
Actual age: 39
Predicted age: 46

1/1 [==============================] - 0s 18ms/step
Actual age: 46
Predicted age: 22

1/1 [==============================] - 0s 19ms/step
Actual age: 2
Predicted age: 11

1/1 [==============================] - 0s 19ms/step
Actual age: 1
Predicted age: 19

1/1 [==============================] - 0s 19ms/step
Actual age: 10
Predicted age: 11

1/1 [==============================] - 0s 19ms/step
Actual age: 18
Predicted age: 18

1/1 [==============================] - 0s 20ms/step
Actual age: 28
Predicted age: 31

1/1 [==============================] - 0s 19ms/step
Actual age: 45
Predicted age: 55

1/1 [==============================] - 0s 21ms/step
Actual age: 11
Predicted age: 28

1/1 [==============================] - 0s 20ms/step
Actual age: 28
Predicted age: 29

1/1 [==============================] - 0s 19ms/step
Actua

In [48]:
# Testing CNN model
test_model(model_cnn, "CNN")

1/1 [==============================] - 0s 165ms/step
Actual age: 45
Predicted age: 35

1/1 [==============================] - 0s 21ms/step
Actual age: 22
Predicted age: 35

1/1 [==============================] - 0s 19ms/step
Actual age: 11
Predicted age: 35

1/1 [==============================] - 0s 19ms/step
Actual age: 55
Predicted age: 35

1/1 [==============================] - 0s 19ms/step
Actual age: 54
Predicted age: 35

1/1 [==============================] - 0s 19ms/step
Actual age: 26
Predicted age: 35

1/1 [==============================] - 0s 18ms/step
Actual age: 21
Predicted age: 35

1/1 [==============================] - 0s 21ms/step
Actual age: 75
Predicted age: 35

1/1 [==============================] - 0s 19ms/step
Actual age: 38
Predicted age: 35

1/1 [==============================] - 0s 20ms/step
Actual age: 26
Predicted age: 35

1/1 [==============================] - 0s 20ms/step
Actual age: 26
Predicted age: 35

1/1 [==============================] - 0s 18ms/step
A

In [ ]:
# Testing AlexNet model
test_model(model_alexnet2, "AlexNet")

1/1 [==============================] - 0s 149ms/step
Actual age: 58
Predicted age: 639168

1/1 [==============================] - 0s 20ms/step
Actual age: 22
Predicted age: 29

1/1 [==============================] - 0s 21ms/step
Actual age: 16
Predicted age: 12074740

1/1 [==============================] - 0s 21ms/step
Actual age: 5
Predicted age: 6124315

1/1 [==============================] - 0s 19ms/step
Actual age: 46
Predicted age: 21

1/1 [==============================] - 0s 21ms/step
Actual age: 26
Predicted age: 1364692

1/1 [==============================] - 0s 21ms/step
Actual age: 22
Predicted age: 1167297

1/1 [==============================] - 0s 21ms/step
Actual age: 24
Predicted age: 28641

1/1 [==============================] - 0s 21ms/step
Actual age: 23
Predicted age: 869521

1/1 [==============================] - 0s 21ms/step
Actual age: 35
Predicted age: 18733076

1/1 [==============================] - 0s 21ms/step
Actual age: 35
Predicted age: 64

1/1 [===========

KeyboardInterrupt: ignored